In [3]:
import os
import random
from sklearn.model_selection import train_test_split
import shutil

# 原始数据集路径
dataset_path = "../datasets/Caltech101"
train_path = "../datasets/Caltech101_train"
test_path = "../datasets/Caltech101_test"

# 创建存放训练集和测试集的文件夹
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# 遍历类别
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    if not os.path.isdir(category_path):
        continue  # 跳过非文件夹

    images = os.listdir(category_path)
    random.shuffle(images)

    # 划分训练集和测试集
    train_images, test_images = train_test_split(images, train_size=30, random_state=42)

    # 复制图片到训练集
    os.makedirs(os.path.join(train_path, category), exist_ok=True)
    for img in train_images:
        shutil.copy(os.path.join(category_path, img), os.path.join(train_path, category, img))

    # 复制图片到测试集
    os.makedirs(os.path.join(test_path, category), exist_ok=True)
    for img in test_images:
        shutil.copy(os.path.join(category_path, img), os.path.join(test_path, category, img))

print("数据集拆分完成！")

数据集拆分完成！


In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cu126
True


In [ ]:
import torch
import torch.nn as nn
import cv2
import numpy as np
from ultralytics import YOLO

# **加载 YOLO11s-Cls 作为 CNN 特征提取器**
yolo_model = YOLO("yolo11cls.pt")

# **多通道输入处理**
def extract_ppt_regions(image):
    """ 从输入图像提取完整图像 + 上/下/左/右 关键区域 """
    h, w, _ = image.shape
    top = image[:h//5, :, :]
    bottom = image[-h//5:, :, :]
    left = image[:, :w//5, :]
    right = image[:, -w//5:, :]
    return image, top, bottom, left, right  # 返回五个区域

def preprocess_image(image):
    """ 预处理输入图像，使其适应 YOLO 训练 """
    image = cv2.resize(image, (640, 640))
    image = image / 255.0  # 归一化
    return torch.tensor(image).permute(2, 0, 1).float()  # (C, H, W)

def prepare_multi_input(image_path):
    """ 读取图像，并提取不同 PPT 关键区域 """
    image = cv2.imread(image_path)
    full, top, bottom, left, right = extract_ppt_regions(image)

    # 处理成 YOLO 可用的输入
    full_t = preprocess_image(full)
    top_t = preprocess_image(top)
    bottom_t = preprocess_image(bottom)
    left_t = preprocess_image(left)
    right_t = preprocess_image(right)

    # **拼接输入（5 个通道）**
    multi_input = torch.cat([full_t, top_t, bottom_t, left_t, right_t], dim=0).unsqueeze(0)  # (1, 15, 640, 640)
    return multi_input

# **测试数据**
multi_input = prepare_multi_input("ppt_screenshot.jpg")
print("Multi-input shape:", multi_input.shape)  # (1, 15, 640, 640)


In [ ]:
print("hello")
